关于提取stp特征信息的测试

In [ ]:
import cadquery as cq
import numpy as np
import trimesh

# 加载 STEP 文件
part = cq.importers.importStep("my_part.stp")

# 获取几何对象
solid = part.val()

# 特征提取
bbox = solid.BoundingBox()
volume = solid.Volume()
surface_area = solid.Area()
center = solid.Center()

# 统计结构复杂度
num_faces = len(solid.Faces())
num_edges = len(solid.Edges())
num_solids = len(part.solids().vals())

# 构建向量
feature_vector = [
    bbox.xlen, bbox.ylen, bbox.zlen,       # 尺寸
    volume, surface_area,                  # 体积+表面积
    center.x, center.y, center.z,          # 重心
    num_faces, num_edges, num_solids       # 拓扑结构
]

print("几何特征向量:", feature_vector)

# STEP 文件导入
shape = cq.importers.importStep("my_part.stp").val()

# 网格化（生成顶点 + 面片）
raw_vertices, raw_faces = shape.tessellate(1.0)

# 将 Vector 类型转为标准 float 数组
vertices = np.array([[v.x, v.y, v.z] for v in raw_vertices], dtype=np.float64)
faces = np.array(raw_faces, dtype=np.int64)

# 构建 trimesh 对象
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# 可视化（可选）
# mesh.show()


几何特征向量: [2200.455276750666, 107.480232008125, 110.5000001, 17817.207283287124, 13850.251743460405, 2253.5905297330523, 181.9699193828728, -7.583479736954674, 155, 367, 1]


In [13]:
import torch
import cadquery as cq
import numpy as np
import trimesh
from simple_autoencoder import PointNetEncoder
# STEP 文件导入
shape = cq.importers.importStep("my_part.stp").val()

# 网格化（生成顶点 + 面片）
# raw_vertices 表示每个顶点的向量
# raw_faces 每个三角形对应的顶点，一种连接关系
raw_vertices, raw_faces = shape.tessellate(1.0)

# 转 numpy
vertices = np.array([[v.x, v.y, v.z] for v in raw_vertices], dtype=np.float64)
faces = np.array(raw_faces, dtype=np.int64)

# 构建 trimesh 对象
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

# 从 mesh 采样 1024 个点
points = mesh.sample(1024)
points = np.array(points, dtype=np.float32)

# 转成 PyTorch Tensor [batch, N, 3]
points_tensor = torch.tensor(points).unsqueeze(0)
print(points_tensor.shape)  # (1, 1024, 3)
print(points_tensor)  # (1, 1024, 3)

encoder = PointNetEncoder(emb_dim=64)

# 输入点云 Tensor (batch=1, N=1024, 3)
out = encoder(points_tensor)
print(out)
print(out.shape)  # (1, 64)  -> 这就是几何 embedding

torch.Size([1, 1024, 3])
tensor([[[2296.3638,  191.2544,   -8.0000],
         [2254.0154,  207.4802,   -8.1927],
         [2255.8584,  181.5598,  -10.5000],
         ...,
         [2277.8291,  159.7005,   -6.6428],
         [2217.3765,  157.3333,   -8.1149],
         [2241.5059,  158.2193,   -7.9073]]])
tensor([[  -6.4373, -145.2244, -169.8031,   72.7563,   54.2577,  -37.0085,
           17.9253,  101.8857,   -5.2086,   26.3775,   30.6453,  -54.6486,
         -126.8712,  146.3412,  -36.9909,   64.2266,   53.4009,   29.2604,
           96.0996,  -55.9712,  208.2715,  -43.6816,  178.7457,   47.1187,
           46.8967,    3.6122,  196.1445,  -14.8343, -109.8733,   53.9591,
           -0.8738,   -0.8278, -167.8523,  -57.6716,  -79.0665,  122.4648,
          -46.7107,    6.9947,   92.7172,   24.7223,  -57.2824,  -33.0520,
          153.2931,  268.6234,  -85.3424,   33.9558,   60.8223,  -93.7649,
           -2.9476,  -21.5788,  -37.1331,   19.1873, -108.9713, -147.3761,
          218.5430, 

关于一个简单的神经网络测试

In [1]:
from simple_autoencoder import VoxelAutoencoder
import torch
# 假设输入数据是 [batch_size, channels, depth, height, width]
x = torch.randn(8, 1, 32, 32, 32)  # batch=8, 单通道, 32^3 体素

model = VoxelAutoencoder(input_channels=1, latent_dim=32)
recon, z = model(x)

print("输入体素:", x.shape)
print("重建体素:", recon.shape)
print("潜在向量:", z.shape)


输入体素: torch.Size([8, 1, 32, 32, 32])
重建体素: torch.Size([8, 1, 32, 32, 32])
潜在向量: torch.Size([8, 32])
